In [ ]:
import csv
import pandas as pd
from owlready2 import *
import csv

In [ ]:
continents_df_final = pd.read_csv('./data/continents_df_final.csv', index_col='Index')
continents_df_final

In [7]:
#if row['Population prospect by 2050']: continent.populationProspect = int(row['Population prospect by 2050'])

In [ ]:
geo_ontology = get_ontology("http://ce.sharif.edu/onto")

ns_continent = get_ontology("http://ce.sharif.edu/onto/continent")
ns_country = get_ontology("http://ce.sharif.edu/onto/country")
ns_region = get_ontology("http://ce.sharif.edu/onto/region")
ns_lang = get_ontology("http://ce.sharif.edu/onto/language")

with geo_ontology:
    class Territory(Thing): pass
    class Continent(Territory): namespace = ns_continent
    class Country(Territory): namespace = ns_country
    class Region(Territory): namespace = ns_region
    class Language(Thing): namespace = ns_lang
    AllDisjoint([Continent, Country, Region, Language])
    

In [ ]:
with geo_ontology:
# Object properties
    class isNeighborWith(Country >> Country): pass
    class hasCountry(Region >> Country): pass
    class region(Country >> Region, FunctionalProperty): inverse_property = hasCountry
    class hasLanguage(Country >> Language): pass
    class hasRegion(Continent >> Region): pass
    class continent(Region >> Continent, FunctionalProperty): inverse_property = hasRegion
    # Data properties
    class population(Territory >> int, FunctionalProperty): pass
    class area(Territory >> int, FunctionalProperty): pass
    class callingCode(Country >> int, FunctionalProperty): pass
    class capital(Country >> str, FunctionalProperty): pass
    class capitalCityPopulation(Country >> int, FunctionalProperty): pass
    class gdp(Country >> float, FunctionalProperty): pass
    class gdpGrowth(Country >> float, FunctionalProperty): pass
    class internetTld(Country >> str, FunctionalProperty): pass
    class iso247(Country >> str, FunctionalProperty): pass
    class fullName(Territory >> str, FunctionalProperty): pass
    class populationDensity(Country >> float, FunctionalProperty): pass
    class populationProspect(Continent >> int, FunctionalProperty): pass
    class sexRatio(Country >> float, FunctionalProperty): pass
    class tzOffset(Country >> str, FunctionalProperty): pass
    class ed_EnergyProduction(Country >> int, FunctionalProperty): pass
    class ed_EnergySupplyPerCapita(Country >> int, FunctionalProperty): pass
    class ed_ForestedArea(Country >> int, FunctionalProperty): pass
    class ed_ImportantSites(Country >> int, FunctionalProperty): pass
    class ed_InternetUsers(Country >> float, FunctionalProperty): pass
    class ed_NetOfficialAssistReceived(Country >> float, FunctionalProperty): pass
    class ed_ThreatenedSpecies(Country >> int, FunctionalProperty): pass
    class sd_CurrentHealthExpenditure(Country >> float, FunctionalProperty): pass
    class sd_EducationExpenditure(Country >> float, FunctionalProperty): pass
    class sd_InfantMortalityRate(Country >> float, FunctionalProperty): pass
    class sd_IntentionalHomicideRate(Country >> float, FunctionalProperty): pass
    class sd_InternationalMigrantStock(Country >> float, FunctionalProperty): pass
    class sd_PhysiciansRatio(Country >> float, FunctionalProperty): pass
    class sd_PopulationGrowthRate(Country >> float, FunctionalProperty): pass
    class sd_PrimaryGrossEnrolRatio(Country >> float, FunctionalProperty): pass
    class sd_RefugeesAndOthers(Country >> float, FunctionalProperty): pass
    class sd_SeatsWomenParliament(Country >> float, FunctionalProperty): pass
    class sd_SecondaryGrossEnrolRatio(Country >> float, FunctionalProperty): pass
    class sd_TertiaryGrossEnrolRatio(Country >> float, FunctionalProperty): pass
    class sd_UrbanPopulation(Country >> float, FunctionalProperty): pass


    for index, row in continents_df_final.iterrows():
        continent =Continent(row['Index.1'], ns_continent, fullName=row['Index.1']) 
        if row['Population(000, 2020)']: continent.population = int(row['Population(000, 2020)'])
        if row['Surface area(km2)']: continent.area = int(row['Surface area(km2)'])
            
geo_ontology.save(file="./output/ontology.owl")